In [112]:
# !pip install nltk


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [113]:
import spacy
import os
import shutil
# import cudf 

import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction import text as for_stop_word
from sklearn.base import BaseEstimator, TransformerMixin
# from cleantext import clean
# import emoji
from collections import Counter
import pickle
from multiprocessing import Pool, current_process, Queue
import cupy as cp

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shpri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
def save_model_pkl(path, model_name):
    with open(path, "wb") as mod:
        pickle.dump(model_name, mod)


def load_model_pkl(path):
    with open(path, "rb") as mod1:
        model_name = pickle.load(mod1)
    return model_name

In [115]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', None)
pd.set_option('max_colwidth', 800)

In [116]:
with open('data/NLP_HW_1_results.pickle', 'rb') as f:
        df = pickle.load(f, encoding='utf-8')

In [117]:
df.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]",father dysfunctional selfish drags kids dysfunction run,father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunction run
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]",thanks lyft credit use cause offer wheelchair vans pdx disapointed getthanked,thank lyft credit use caus offer wheelchair van pdx disapoint getthank,thanks lyft credit use cause offer wheelchair van pdx disapointed getthanked
2,3,0.0,bihday your majesty,"[bihday, your, majesty]",bihday majesty,bihday majesti,bihday majesty
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, time, in, ur]",model love take time ur,model love take time ur,model love take time ur
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]",factsguide society motivation,factsguid societi motiv,factsguide society motivation


In [118]:
# !pip install "spacy[cuda11x]" pygls>=1.0.0

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\shpri\\miniconda3\\envs\\envsgb39\\Lib\\site-packages\\cupy\\_util.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [126]:
import spacy
from spacy import displacy
spacy.prefer_gpu()

True

In [127]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

In [128]:
corpus = " ".join(df['tweet_lemmatized'])
corpus = corpus[:999999]

In [122]:
corpus[-25:]

'se claim right rough demo'

In [129]:
doc = nlp(corpus)

  0%|                                                                                  | 0/145621 [03:54<?, ?it/s]


In [ ]:
displacy.render(doc, style="ent")

In [131]:
ents_dict = {}
for entity in tqdm(doc.ents):
    if entity.label_ in ents_dict:
        ents_dict[entity.label_] += 1
    else:
        ents_dict[entity.label_] = 1

ents_dict

100%|███████████████████████████████████████████████████████████████████| 11840/11840 [00:00<00:00, 910434.48it/s]


{'PERSON': 1833,
 'ORG': 917,
 'DATE': 4152,
 'GPE': 1391,
 'NORP': 594,
 'CARDINAL': 1578,
 'TIME': 721,
 'ORDINAL': 377,
 'LOC': 62,
 'QUANTITY': 54,
 'FAC': 38,
 'PRODUCT': 74,
 'WORK_OF_ART': 3,
 'LAW': 4,
 'MONEY': 12,
 'PERCENT': 9,
 'LANGUAGE': 13,
 'EVENT': 8}

In [132]:
token_dict = {}
for token in tqdm(doc):
    if token.pos_ in token_dict:
        token_dict[token.pos_] += 1
    else:
        token_dict[token.pos_] = 1

token_dict

100%|█████████████████████████████████████████████████████████████████| 145621/145621 [00:00<00:00, 927454.31it/s]


{'PROPN': 30990,
 'NOUN': 55600,
 'VERB': 23923,
 'ADJ': 20256,
 'ADV': 5112,
 'AUX': 1455,
 'ADP': 1585,
 'INTJ': 1399,
 'NUM': 2656,
 'PRON': 1416,
 'SCONJ': 375,
 'DET': 345,
 'PART': 266,
 'PUNCT': 18,
 'X': 142,
 'CCONJ': 62,
 'SPACE': 21}

In [133]:
save_model_pkl("cpModel.pkl", doc)

In [134]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shpri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [135]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\shpri\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [136]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\shpri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     C:\Users\shpri\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!


True

In [ ]:
chunk_corpus = []
for sent in tqdm(nltk.sent_tokenize(corpus)):
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
     if hasattr(chunk, 'PERSON'):
        chunk_corpus.append((chunk.label(), ' '.join(c[0] for c in chunk)))

  0%|                                                                                       | 0/1 [00:00<?, ?it/s]

In [ ]:
chunk_corpus[:35]

In [ ]:
chunk_dict = {}
for chunk in tqdm(chunk_corpus):
    if chunk[0] in chunk_dict:
        chunk_dict[chunk[0]] += 1
    else:
        chunk_dict[chunk[0]] = 1

chunk_dict

In [ ]:
chunk_dict = {}
for chunk in chunk_corpus:
    if chunk[0] in chunk_dict:
        chunk_dict[chunk[0]] += 1
    else:
        chunk_dict[chunk[0]] = 1

chunk_dict

In [ ]:
# TOP-20 популярных личностей
person_tokens_list = [(ent[0], ent[1]) for ent in chunk_corpus if ent[0] == 'PERSON']
Counter(person_tokens_list).most_common(20)

In [ ]:
# TOP-20 популярных организаций
org_tokens_list = [(ent[0], ent[1]) for ent in chunk_corpus if ent[0] == 'ORGANIZATION']
Counter(org_tokens_list).most_common(20)